<a href="https://colab.research.google.com/github/Shashank-Shukla/PyTorch-practice/blob/master/PyTorch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
%matplotlib inline

print("Torch version: ",torch.__version__)
print("Cuda active: ",torch.cuda.is_available())

Torch version:  1.6.0+cu101
Cuda active:  True


In [2]:
'''
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x
'''

'\nclass NN(nn.Module):\n  def __init__(self, input_size, num_classes):\n    super(NN, self).__init__()\n    self.fc1 = nn.Linear(input_size, 50)\n    self.fc2 = nn.Linear(50, num_classes)\n\n  def forward(self, x):\n    x = F.relu(self.fc1(x))\n    x = self.fc2(x)\n    return x\n'

In [3]:
class CNN(nn.Module):
  def __init__(self, in_channels=1, num_classes=10):
    # for mnmist dataset, in_channel = 1 and num_classes = 10
    super(CNN, self).__init__()
    # same convolution: kernel=3x3 and padding=1x1 and stride=1x1
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
    # n_out = ((n_in + 2*pad - kernal)/stride) + 1
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.fc1 = nn.Linear(16*7*7, num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)
    return x
  

In [4]:
model = CNN()
x = torch.randn(64,1,28,28)
print("X shape = ",x.shape)
print("Model shape: ",model(x).shape)


X shape =  torch.Size([64, 1, 28, 28])
Model shape:  torch.Size([64, 10])


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
in_channel = 1
num_classes = 10
lr = 0.001
bs = 64
epochs = 10

# Loading data train and test respectively
train_dataset = datasets.MNIST(root='dataset/',train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/',train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=True)

# Initialize model
model = CNN().to(device)

# Setting loss and optimizer criterion
criterion_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training the network
for epoch in range(epochs):
  losses = []
  for batch_idx, (data, targets) in enumerate(train_loader):
    # Run in cuda
    data = data.to(device)
    targets = targets.to(device)

    # Forward propagation
    scores = model(data)
    loss = criterion_loss(scores, targets)
    losses.append(loss.item())

    # Backward propagation
    optimizer.zero_grad()
    loss.backward()

    # Gradient descent
    optimizer.step()
  mean_loss = sum(losses)/len(losses)
  #scheduler.step(mean_loss)
  print(f"Accuray at epoch {epoch}: {1-mean_loss:.2f}")

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Accuray at epoch 0: 0.66
Accuray at epoch 1: 0.89
Accuray at epoch 2: 0.92
Accuray at epoch 3: 0.93
Accuray at epoch 4: 0.94
Accuray at epoch 5: 0.95
Accuray at epoch 6: 0.95
Accuray at epoch 7: 0.96
Accuray at epoch 8: 0.96
Accuray at epoch 9: 0.97


In [27]:
def check_acc(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training-set")
  else:
    print("Checking accuracy on testing-set")
  num_correct = 0
  num_sample = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x = x.to(device)

      scores = model(x)
      _, preds = scores.max(1)
      num_correct += (preds == y).sum()
      num_sample += preds.size(0)

    print(f"Got {num_correct}/{num_sample} = {float(num_correct)/num_sample * 100:.2f}")

  model.train()


In [28]:
check_acc(train_loader, model)
check_acc(test_loader, model)

Checking accuracy on training-set
Got 58413/60000 = 97.36
Checking accuracy on testing-set
Got 9754/10000 = 97.54
